In [1]:
import pandas as pd
import numpy as np

In [2]:
# reading csv's
school_data = pd.read_csv('Resources/schools_complete.csv')
student_data = pd.read_csv('Resources/students_complete.csv')
school_df = pd.merge(student_data,school_data,how ='left',on='school_name')
#total no of schools
school_total = school_df.school_name.value_counts().count()
#total no of students
student_total = school_df["Student ID"].count()
# total budget
#-changed it at the end instead---total_budget= school_data.budget.sum().astype(str)
total_budget= school_data.budget.sum()
#Average math score
avg_math = school_df.math_score.mean()
#Average reading score
avg_read= school_df.reading_score.mean()
#overall passing rate
overall_pass = (school_df.math_score.mean() + school_df.reading_score.mean())/2
# percentage of student with passing math score 70 or greater
new_df = (school_df.loc[school_df.math_score >=70])
per_math=new_df.student_name.count()/school_df.student_name.count()*100
 # percentage of student with passing reading score 70 or greater
new_df = (school_df.loc[school_df.reading_score >=70])
per_reading=new_df.student_name.count()/school_df.student_name.count()*100 
#District DataFrame
district_df =pd.DataFrame(columns =['Total\n Schools','Total\nStudents','Total Budget','Average\nMath\nScore','Average\nReading\nScore','%\nPassing\nMath','%\nPassing\nReading','%\nOverall\Passing'],data=[[school_total,student_total,total_budget,avg_math,avg_read,per_math,per_reading,overall_pass]])
district_df['Total Budget']='$' + district_df['Total Budget'].map('{:,}'.format)

#the Region Summary
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall\Passing
0,15,39170,"$24,649,428",78.985371,81.87784,74.980853,85.805463,80.431606


In [3]:
# grouped by school name  
new_df = school_df.groupby('school_name')
# create a dataframe and count no of students
school_summary = pd.DataFrame(new_df['student_name'].count())
# creating dateframe for school_type
school_type =pd.DataFrame(new_df['type'].unique())
student_count=new_df.student_name.count()
school_summary1 = pd.merge(school_summary,school_type,how='left', on='school_name')

# total school budget
budget_df = pd.DataFrame(new_df['budget'].unique())
school_summary = pd.merge(school_summary1,budget_df,how='left',on = 'school_name')

#budget per student
total_sc_budget_per = new_df['budget'].unique().values/student_count
total_sc_df = total_sc_budget_per.to_frame()
school_summary2 = pd.merge(school_summary,total_sc_df,how ='left',on='school_name')
school_summary2.rename(columns={'student_name_x':'Total\nStudents','student_name_y':'Per\nStudent\nBudget'},inplace = True)

#calculate average math score
avg_math=school_df.groupby('school_name')['math_score'].mean()
new_avg_math=avg_math.to_frame()
new_avg_math.rename(columns={'math_score':'Average\nMath\nScore'},inplace=True)
school_summary3 = pd.merge(school_summary2,new_avg_math,how ='left',on='school_name')

#calculate average reading score
avg_reading=school_df.groupby('school_name')['reading_score'].mean()
new_avg_reading=avg_reading.to_frame()
new_avg_reading.rename(columns={'reading_score':'Average\nReading\nScore'},inplace = True)
school_summary4 = pd.merge(school_summary3,new_avg_reading,how ='left',on='school_name')

#computing pass % math
math_stu_pass= school_df[school_df['math_score']>= 70].groupby('school_name')['student_name'].count()
total_stu_school=school_df.groupby('school_name')['student_name'].count()
math_per_pass = math_stu_pass/total_stu_school*100
math_per_pass_df = math_per_pass.to_frame()
math_per_pass_df.rename(columns={'student_name':'%Passing\nMath'},inplace =True)
school_summary5=pd.merge(school_summary4,math_per_pass_df,how='left',on='school_name')

#computing pass % reading
reading_stu_pass= school_df[school_df['reading_score']>= 70].groupby('school_name')['student_name'].count()
reading_per_pass = reading_stu_pass/total_stu_school*100
reading_per_pass_df = reading_per_pass.to_frame()
reading_per_pass_df.rename(columns={'student_name':'%Passing\nReading'},inplace =True)
school_summary6=pd.merge(school_summary5,reading_per_pass_df,how='left',on='school_name')

#overall passing rate
over_pass_rate =(math_per_pass+reading_per_pass)/2
overall_pass_rate=over_pass_rate.to_frame()
overall_pass_rate.rename(columns={'student_name':'Overall Pass\n Rate'},inplace = True)
school_summary7=pd.merge(school_summary6,overall_pass_rate,how='left',on ='school_name')

#cleaning the data(removing brackets)
school_summary7['type']=school_summary7['type'].str.get(0)
school_summary7['budget']=school_summary7['budget'].str.get(0)
school_summary7.rename(columns={'type':'School\nType','budget':'Total\nSchool\nBudget'},inplace=True)
school_summary7['Per\nStudent\nBudget']=school_summary7['Per\nStudent\nBudget'].str.get(0)
school_summarydf=school_summary7.sort_values(by=['Overall Pass\n Rate'],ascending = False)

school_summarydf['Total\nSchool\nBudget']= school_summarydf['Total\nSchool\nBudget'].map('${:,}'.format)
school_summarydf['Per\nStudent\nBudget']= school_summarydf['Per\nStudent\nBudget'].map('${:.1f}'.format)

#top performing
school_summarydf.head(5)

,Total Students,School Type,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate
school_name,,,,,,,,,
Cabrera High School,1858,Charter,"$1,081,356",$582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,1635,Charter,"$1,043,130",$638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,962,Charter,"$585,858",$609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,1468,Charter,"$917,500",$625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,2283,Charter,"$1,319,574",$578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [4]:
#bottom performing
school_summarydf=school_summary7.sort_values(by=['Overall Pass\n Rate'],ascending = True)

school_summarydf['Total\nSchool\nBudget']= school_summarydf['Total\nSchool\nBudget'].map('${:,}'.format)
school_summarydf['Per\nStudent\nBudget']= school_summarydf['Per\nStudent\nBudget'].map('${:.1f}'.format)
school_summarydf.head(5)

,Total Students,School Type,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate
school_name,,,,,,,,,
Rodriguez High School,3999,District,"$2,547,363",$637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,2949,District,"$1,884,411",$639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,2917,District,"$1,910,635",$655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,4761,District,"$3,094,650",$650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,2739,District,"$1,763,916",$644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [5]:
#math avg by grades
avg_math=pd.pivot_table(school_df,index="school_name",columns="grade",values="math_score",aggfunc="mean")
avg_math=avg_math[["9th","10th","11th","12th"]]
avg_math


grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [6]:
#Reading average  by grades
avg_reading=pd.pivot_table(school_df,index="school_name",columns="grade",values="reading_score",aggfunc="mean")
avg_reading=avg_reading[["9th","10th","11th","12th"]]
avg_reading

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [7]:
#binning by scores
new_bin=school_summary7.reset_index()
score_bin=new_bin
#creating bins
bins =[0,585,615,645,675]
#creating names for the bin
groups =["<$585","$585-615","$615-645","$645-675"]
score_bin["Spending Ranges\n(Per Student)"] =pd.cut(score_bin['Per\nStudent\nBudget'],bins,labels=groups)
score_bin = score_bin.groupby(score_bin["Spending Ranges\n(Per Student)"]).agg({
                                                            'Average\nMath\nScore':'mean',
                                                            'Average\nReading\nScore':"mean",
                                                            '%Passing\nMath':"mean",
                                                            '%Passing\nReading':"mean",
                                                            'Overall Pass\n Rate':"mean"

})
score_bin


,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [8]:
#Scores by school size
school_size =pd.DataFrame(new_df['size'].unique())
school_size_df=pd.merge(school_summary7,school_size,how='left',on='school_name')
size_bin=school_size_df.reset_index()
#creating bins
bins=[0,1000,2000,5000]
groups=["small (<10000)","Medium (1000-2000)","Large (2000-3000)"]
size_bin["Scores by\nSchool Size"]=pd.cut(size_bin['size'],bins,labels=groups)
size_bin=size_bin.groupby(size_bin["Scores by\nSchool Size"]).agg({
                                            'Average\nMath\nScore':"mean",
                                            'Average\nReading\nScore':"mean",
                                            '%Passing\nMath':"mean",
                                            '%Passing\nReading':"mean",
                                            'Overall Pass\n Rate':"mean"
})
size_bin

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate
Scores by School Size,,,,,
small (<10000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-3000),77.746417,81.344493,69.963361,82.766634,76.364998


In [9]:
#Score by school Type
type_bin=new_bin.drop(['school_name','Total\nStudents','Total\nSchool\nBudget','Per\nStudent\nBudget'],axis= 1)
new_type_df=type_bin.groupby(['School\nType'])['Average\nMath\nScore','Average\nReading\nScore','%Passing\nMath','%Passing\nReading','Overall Pass\n Rate'].mean()
new_type_df

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [ ]:
School Analysis:

In top performing schools the student budget is below the region budget.
Average number of students per school also is below the region average as opposed to the bottom performing schools, where both are higher.
Charter schools are better performers.
It seems more budget does not translate to better performance.

More Data is needed  before we come to the conclusion that less students mean better grades.
Missing Data:
    1. The low income students in the schools- that could be the reason for more spending per student(for lunch)
    2. The crime rate in the area of the school.
    3. Ethenic diversity of the school